In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Steps  
1. Import libraries  
2. Prepare data  
   ```Download  |  Transform  |  Dataloader```  
3. Define parameters  
   ```Model  |  Optimizer  |  Loss  |  Training  ```
4. Build Model  
   ```Components  ```
5. Training loop  
6. Visualize results  

In [2]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F    # New import
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import time
import datetime

print(f"Imports completed at {datetime.datetime.now()}")

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Versions
print(f"Torch: {torch.__version__}, TorchVision: {torchvision.__version__}")

Imports completed at 2025-08-22 18:56:31.990100
Using device: cuda
Torch: 2.6.0+cu124, TorchVision: 0.21.0+cu124


In [4]:
# Set seed for PyTorch
seed = 42
torch.manual_seed(seed)

# Data prep params
batch_size = 128

# Model params
z_dim = 100     # Size of noise vector
img_dim = 28       # MNIST size
channels = 1        # Gray scale
img_shape = (channels, img_dim, img_dim) # MNIST shape

# Optimizer params
learning_rate = 0.0002
# beta1 = 0.5  # Adam optimizer beta1

# # Loss params
# criterion = nn.BCELoss()

# Training params
num_epochs = 100
generator_rounds = 1        # Note we train generator less than critic in WGAN
critic_rounds = 1           # Note we use name critic (not discriminator) in WGAN. It is deliberate.
clip_value = 0.01

In [55]:
# Transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])
print(f"Transform to be applied:\n{transform}\n")

# Load MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
print(f"Train data:\n{train_dataset}\n")

# Dataloader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True) # Last batch will be 96 (6000 % 128) instead of 128, so drop to avoid problem with batch norm
print(f"Data loader:\n{train_loader}")

Transform to be applied:
Compose(
    ToTensor()
    Normalize(mean=(0.5,), std=(0.5,))
)

Train data:
Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

Data loader:


In [6]:
dz = 128
z_fixed = torch.randn(64, dz, device=device)
z_fixed

tensor([[ 0.1940,  2.1614, -0.1721,  ..., -1.2072, -0.2438, -0.6784],
        [ 0.1973,  0.9782, -0.0287,  ..., -0.9226,  0.7893, -1.8010],
        [-1.9351,  1.1007,  0.0251,  ..., -0.6872, -0.0919, -0.7132],
        ...,
        [-1.0439,  0.7082,  0.0871,  ..., -0.2350,  0.6789, -0.2683],
        [ 0.1728,  0.9311,  0.9508,  ..., -1.2910, -0.3644,  0.9805],
        [-0.5130,  0.6719, -0.1551,  ..., -1.0449,  1.5397, -2.0042]],
       device='cuda:0')

## Model   

## Optimizers  

## Training loop  

## Training  

## Questions  

### E1  

In [37]:
train_dataset.classes

['0 - zero',
 '1 - one',
 '2 - two',
 '3 - three',
 '4 - four',
 '5 - five',
 '6 - six',
 '7 - seven',
 '8 - eight',
 '9 - nine']

In [40]:
counts = torch.bincount(train_dataset.targets)
counts

tensor([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949])

In [43]:
torch.max(counts)

tensor(6742)

In [44]:
torch.max(counts)

tensor(5421)

In [67]:
(torch.max(counts) - torch.min(counts)) / counts.float().mean() * 100

tensor(22.0167)

### E2  

In [60]:
import torch
from torch.utils.data import DataLoader

# Use your already-defined train_dataset (with ToTensor + Normalize((0.5,), (0.5,)))
loader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)

# Grab exactly the first batch after transforms
X, _ = next(iter(loader))              # X shape: [B, C, H, W], dtype float32 in [-1, 1]

# Compute stats on the batch tensor
mu = X.mean().item()
sigma = X.std(unbiased=False).item()   # population std to match spec-style reporting
m = X.min().item()
M = X.max().item()

# p = fraction of pixels with |x| <= 0.5
p = (X.abs() <= 0.5).float().mean().item()

print( round(mu, 3), round(sigma, 3), round(m, 3), round(M, 3), round(p, 3) )
# -> (µ, σ, m, M, p)


-0.742 0.613 -1.0 1.0 0.05
